In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.formula.api as sm
import statsmodels.api as stats
from ast import literal_eval

> ## Clean and Sort Data
>
> 
> In this project we have compiled two movie datasets. One contains **Top 100 movies** from each year from 2014-2025. The other Dataframe contains **best picture movies** from wikipedia. We will use these datasets to gather valuable insights. In this notebook I merge the datasets together and do some necessary cleaning so that we can effectivley explore genres als CPI adjusted **budget and revenue** values.
>
> 
>* **Merge** and save data.
>* Add **adjusted CPI** budget and revenue columns
>* **Expand genres** column for exploration
>
>

In [2]:
# read in csv files
best_picture = pd.read_csv('../data/best_picture.csv')
movies_2015_2024 = pd.read_csv('../data/movies_2015_2024.csv')

In [3]:
# inspect dataframe
best_picture.shape

(611, 3)

In [4]:
# inspect dataframe
movies_2015_2024.shape

(1000, 9)

In [5]:
best_picture.head(2)

,Year,Film,Winner
0,1927,Wings,Yes
1,1927,7th Heaven,No


In [6]:
movies_2015_2024.head(2)

,title,vote_average,vote_count,budget,revenue,imdb_id,id,movie_genres,release_year
0,#Alive,7.227,1955,6300000,13416285,tt10620868,614696,"['Action', 'Horror', 'Science Fiction']",2020
1,10 Cloverfield Lane,6.994,8359,15000000,110216998,tt1179933,333371,"['Thriller', 'Science Fiction', 'Drama', 'Horr...",2016


In [7]:
# rename columns
best_picture_rename = best_picture.rename(columns={
    "Year": "release_year",
    "Film": "title"
})

# merge dataframes and keep all movies
movies_merged = movies_2015_2024.merge(
    best_picture_rename[["release_year", "title", "Winner"]],
    on=["release_year", "title"],
    how="left"
)

# fill in missing winner values with no
movies_merged["Winner"] = movies_merged["Winner"].fillna("No")

# display merged dataframe
movies_merged.head(2)

,title,vote_average,vote_count,budget,revenue,imdb_id,id,movie_genres,release_year,Winner
0,#Alive,7.227,1955,6300000,13416285,tt10620868,614696,"['Action', 'Horror', 'Science Fiction']",2020,No
1,10 Cloverfield Lane,6.994,8359,15000000,110216998,tt1179933,333371,"['Thriller', 'Science Fiction', 'Drama', 'Horr...",2016,No


In [8]:
# export data to csv file
movies_merged.to_csv('movies_merged.csv', index=False)

> ## CPI Adjustment
>
> 
> In this project we have collected **revenue and budget data** for multiple years. In order to substansiate that growth by **todays financial landscape** we must adjust each individual value. We can make these changes by adjusting for **CPI growth**. Below, I have added a calculation to create a multiplier variable. This can be applied to each year and each value. These average CPI values come from [The Federal Reserve of Minneapolis](https://www.minneapolisfed.org/about-us/monetary-policy/inflation-calculator/consumer-price-index-1913-) **Last Acessed: November 19, 2025**
>
> 
>* Make changes that account for **inflation**.
>* Calculate CPI adjustments.
>* Create **adjusted budget and revenue** columns.

In [9]:
# create copy of merged data for cpi calculations
merged_cpi = movies_merged

In [10]:
# calculate CPI multipliers based off 2024 CPI
cpi_calculation = {
    2015: (313.7 / 237.0),
    2016: (313.7 / 240.0),
    2017: (313.7 / 245.1),
    2018: (313.7 / 251.1),
    2019: (313.7 / 255.7),
    2020: (313.7 / 258.8),
    2021: (313.7 / 271.0),
    2022: (313.7 / 292.7),
    2023: (313.7 / 304.7),
    2024: (313.7 / 313.7),
}

# loop through list and round each value
for year, value in cpi_calculation.items():
    print(year, f"{value:.2f}")

2015 1.32
2016 1.31
2017 1.28
2018 1.25
2019 1.23
2020 1.21
2021 1.16
2022 1.07
2023 1.03
2024 1.00


In [11]:
# create multiplier element for mapping columns
cpi_multiplier = {
    2015: 1.32,
    2016: 1.31,
    2017: 1.28,
    2018: 1.25,
    2019: 1.23,
    2020: 1.21,
    2021: 1.16,
    2022: 1.07,
    2023: 1.03,
    2024: 1.00
}

In [12]:
# map multiplier values in new column
merged_cpi['cpi_multiplier'] = merged_cpi['release_year'].map(cpi_multiplier)

# add adjusted CPI revenue and budget columns
merged_cpi['budget_2024'] = merged_cpi['budget'] * merged_cpi['cpi_multiplier']
merged_cpi['revenue_2024'] = merged_cpi['revenue'] * merged_cpi['cpi_multiplier']

# display adjusted dataframe
merged_cpi.head()

,title,vote_average,vote_count,budget,revenue,imdb_id,id,movie_genres,release_year,Winner,cpi_multiplier,budget_2024,revenue_2024
0,#Alive,7.227,1955,6300000,13416285,tt10620868,614696,"['Action', 'Horror', 'Science Fiction']",2020,No,1.21,7623000.0,1.623370e+07
1,10 Cloverfield Lane,6.994,8359,15000000,110216998,tt1179933,333371,"['Thriller', 'Science Fiction', 'Drama', 'Horr...",2016,No,1.31,19650000.0,1.443843e+08
2,12 Strong,6.300,3096,35000000,67450815,tt1413492,429351,"['War', 'Drama', 'Action', 'History']",2018,No,1.25,43750000.0,8.431352e+07
3,13 Hours: The Secret Soldiers of Benghazi,7.269,3789,50000000,69411370,tt4172430,300671,"['War', 'Action', 'History', 'Drama', 'Thriller']",2016,No,1.31,65500000.0,9.092889e+07
4,1917,7.986,13091,100000000,446064352,tt8579674,530915,"['War', 'History', 'Drama', 'Action']",2019,No,1.23,123000000.0,5.486592e+08


In [13]:
# export adjusted CPI csv file
merged_cpi.to_csv('merged_cpi.csv', index=False)

In [14]:
# create copy to explode
merged_copy = movies_merged

In [15]:
# convert column to list for explode
merged_copy['movie_genres'] = merged_copy['movie_genres'].apply(literal_eval)

In [16]:
# expand genres and display dataframe
merged_genre_explode = merged_copy.explode('movie_genres')
merged_genre_explode.head()

,title,vote_average,vote_count,budget,revenue,imdb_id,id,movie_genres,release_year,Winner,cpi_multiplier,budget_2024,revenue_2024
0,#Alive,7.227,1955,6300000,13416285,tt10620868,614696,Action,2020,No,1.21,7623000.0,1.623370e+07
0,#Alive,7.227,1955,6300000,13416285,tt10620868,614696,Horror,2020,No,1.21,7623000.0,1.623370e+07
0,#Alive,7.227,1955,6300000,13416285,tt10620868,614696,Science Fiction,2020,No,1.21,7623000.0,1.623370e+07
1,10 Cloverfield Lane,6.994,8359,15000000,110216998,tt1179933,333371,Thriller,2016,No,1.31,19650000.0,1.443843e+08
1,10 Cloverfield Lane,6.994,8359,15000000,110216998,tt1179933,333371,Science Fiction,2016,No,1.31,19650000.0,1.443843e+08


In [17]:
# exort genre explode data to csv file
merged_genre_explode.to_csv('merged_genre_explode.csv', index=False)